In [2]:
#pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [3]:
#pip uninstall mex -y

In [164]:
import numpy as np
import timeit
import scipy.io as sio
import datetime

# mex
from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, \
find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, \
constrain, add_obj, obj, maxz, minz

# scipy
from pytest import approx
from scipy.optimize import linprog

In [8]:
from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, \
find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, \
constrain, add_obj, obj, maxz, minz

In [5]:
def generate_tableau(A, b, blow, bupp, c, compr = True):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Bajo el problema del tipo:
    min/max c^Tx
    Ax = b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    
    if compr:
        A = A.todense()
        A = np.array(A)
    
    n_restr, n_vars = A.shape

    # Concatenar A con matrices identidades de variables sinteticas de forma rbind
    identidad_vars_sint = np.eye(n_vars, n_vars)
    c = c.T
    A_new = np.concatenate((A, identidad_vars_sint, -identidad_vars_sint, c), axis=0)

    # Concatenar variables coeficientes sinteticas
    A_sint = np.eye(n_restr + 2*n_vars + 1, n_restr + 2*n_vars + 1)
    A_sint[0:n_restr,:] = A_sint[0:n_restr,:]*0
    
    # Concatenar A_new con A_sint de formca rbind
    A_new = np.c_[A_new, A_sint]
    
    # Juntar cotas de restricciones
    b_new = np.concatenate((b, bupp, -blow, np.array([[0]])), axis=0)
    
    # Concatenar A_new, b_new de forma cbind
    A_new = np.c_[A_new, b_new]
    
    return A_new

In [144]:
def generate_tableau(A, b, blow, bupp, c, compr = True):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Bajo el problema del tipo:
    min/max c^Tx
    Ax = b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    
    if compr:
        A = A.todense()
        A = np.array(A)
    
    n_restr, n_vars = A.shape
    
    
    # Caso en que blow = 0, esto es, no es ninguna restriccion bajo los supuestos de simplex
    if (blow.sum() == 0) and (min(blow)==0) and False:
        # Concatenar A con matrices identidades de variables sinteticas de forma rbind
        identidad_vars_sint = np.eye(n_vars, n_vars)
        c = c.T
        A_new = np.concatenate((A, identidad_vars_sint, c), axis=0)
    
        # Concatenar variables coeficientes sinteticas
        A_sint = np.eye(n_restr + 1*n_vars + 1, n_restr + 1*n_vars + 1)
        A_sint[0:n_restr,:] = A_sint[0:n_restr,:]*0
        
        # Concatenar A_new con A_sint de formca rbind
        A_new = np.c_[A_new, A_sint]
        
        # Juntar cotas de restricciones
        b_new = np.concatenate((b, bupp, np.array([[0]])), axis=0)
        
        # Concatenar A_new, b_new de forma cbind
        A_new = np.c_[A_new, b_new]
    
    else:
        # Concatenar A con matrices identidades de variables sinteticas de forma rbind
        identidad_vars_sint = np.eye(n_vars, n_vars)
        c = c.T
        A_new = np.concatenate((A, identidad_vars_sint, -identidad_vars_sint, c), axis=0)
    
        # Concatenar variables coeficientes sinteticas
        A_sint = np.eye(n_restr + 2*n_vars + 1, n_restr + 2*n_vars + 1)
        A_sint[0:n_restr,:] = A_sint[0:n_restr,:]*0
        
        # Concatenar A_new con A_sint de formca rbind
        A_new = np.c_[A_new, A_sint]
        
        # Juntar cotas de restricciones
        b_new = np.concatenate((b, bupp, -blow, np.array([[0]])), axis=0)
        
        # Concatenar A_new, b_new de forma cbind
        A_new = np.c_[A_new, b_new]
    
    return A_new

# Prueba de función `generate_tableau`

In [145]:
matrix_min_approx = create_matrix(2,7)
matrix_min_approx

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [146]:
constrain(matrix_min_approx,'1,1,E,6')
constrain(matrix_min_approx,'-1,2,E,6')
constrain(matrix_min_approx,'4,3,E,6')
constrain(matrix_min_approx,'1,0,G,0')
constrain(matrix_min_approx,'0,1,G,0')
constrain(matrix_min_approx,'1,0,L,18')
constrain(matrix_min_approx,'0,1,L,18')
obj(matrix_min_approx,'-1,-3,0')
matrix_min_approx

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 18.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [147]:
A_prueba = np.array([[1,1], [-1,2], [4,3]])
b_prueba = np.array([[6,6,6]]).T
c_prueba = np.array([[1,3]]).T
blow_prueba = np.array([[0,0]]).T
bupp_prueba = np.array([[18,18]]).T

In [179]:
tableau_prueba = generate_tableau(A_prueba, b_prueba, blow_prueba, bupp_prueba, c_prueba, False)
tableau_prueba

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., 18.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

# Prueba del paquete `mex`

In [149]:
#mat = sio.loadmat('../data/AFIRO.mat')
mat = sio.loadmat('../data/25fv47.mat')

In [150]:
A = mat['A']
b = mat['b']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
c = mat['c']

In [151]:
min(b)

array([-160.])

In [152]:
b.sum()

30235.929816999997

In [153]:
ubounds.sum()

1.8759999999999997e+35

In [154]:
lbounds.sum()

0

In [155]:
min(lbounds)

array([0], dtype=uint8)

In [156]:
print(A.shape)

(821, 1876)


In [157]:
tableau = generate_tableau(A, b, lbounds, ubounds, c)

In [158]:
print("Inicio a correr a las: ")
now = datetime.datetime.now()
print(now.year,"-", now.month,"-", now.day,":\_o_/ Hora \_o_/:", now.hour,":", now.minute,":", now.second)

Inicio a correr a las: 
2021 - 5 - 15 :\_o_/ Hora \_o_/: 13 : 5 : 47


In [160]:
problem_approx = minz(tableau)

Valor objetivo 0.0
##################################
 Terminó pivoteo de columnas

Valor objetivo 19.2
Valor objetivo 919.2
Valor objetivo 1719.2
Valor objetivo 1959.2
Valor objetivo 2309.2
Valor objetivo 2959.2
Valor objetivo 3559.2
Valor objetivo 3699.6454999999996
Valor objetivo 3773.3614999999995
Valor objetivo 3850.1489999999994
Valor objetivo 3874.7209999999995
Valor objetivo 3890.0784999999996
Valor objetivo 3892.358591954023
Valor objetivo 3898.0855062972587
Valor objetivo 3899.4827078823587
##################################
 Terminó pivoteo de renglones

Tiempo de codigo:  -0.02381351300005008


In [162]:
problem_approx['min']

-3899.4827078823587

In [196]:
tableau_prueba = tableau

In [200]:
n_restr = tableau_prueba.shape[0]-1
n_vars = A.shape[1]

In [201]:
n_restr

4573

In [202]:
c_min_obj = tableau_prueba[-1,0:n_vars]
print(c_min_obj)

[0.     0.     0.     ... 0.     0.     1.5313]


In [203]:
A_min_obj = tableau_prueba[0:n_restr, 0:n_vars]
print(A_min_obj)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [-0. -0. -0. ... -1. -0. -0.]
 [-0. -0. -0. ... -0. -1. -0.]
 [-0. -0. -0. ... -0. -0. -1.]]


In [204]:
b_min_obj = tableau_prueba[0:n_restr, -1]
print(b_min_obj)

[ 0. 29. 60. ...  0.  0.  0.]


In [205]:
c_min_obj.shape

(1876,)

In [206]:
A_min_obj.shape

(4573, 1876)

In [207]:
b_min_obj.shape

(4573,)

In [208]:
min_obj = linprog(c_min_obj, A_ub = A_min_obj, b_ub = b_min_obj).fun
coeff_obj = linprog(c_min_obj, A_ub = A_min_obj, b_ub = b_min_obj).x

KeyboardInterrupt: 